<a href="https://colab.research.google.com/github/Monisha2604/Javascript/blob/master/DT_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:

from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('sdss').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/processed_redshift.csv")
# data=spark.read.csv("/content/drive/MyDrive/MyTable_13_snigdha.csv",inferSchema=True,header=True)
df.show()

KeyboardInterrupt: ignored

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator


In [ ]:
from sklearn.tree import DecisionTreeRegressor 
  
  
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  

In [ ]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=10)

In [ ]:
import time
start_time = time.process_time()
regressor.fit(x_train, y_train)
end_time = time.process_time()

delta_time = end_time - start_time
print(f'run-time: {round(delta_time/60.0, 2)}')

run-time: 1.97


In [ ]:
y_pred = regressor.predict(x_test)

NameError: ignored

In [ ]:
from sklearn import metrics
print(metrics.r2_score(y_test,y_pred))

NameError: ignored

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

In [ ]:
transformed= transData(data)
transformed.show(5)

+--------------------+------------------+
|            features|             label|
+--------------------+------------------+
|[24.35021,22.7895...|0.8162744000000001|
|[24.47392,22.4341...|         0.5564747|
|[23.38084,22.5427...|          0.619608|
|[24.7225799999999...|         0.4611808|
|[25.2249,24.46752...|         0.6421724|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures").fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)


+--------------------+------------------+--------------------+
|            features|             label|     indexedFeatures|
+--------------------+------------------+--------------------+
|[24.35021,22.7895...|0.8162744000000001|[24.35021,22.7895...|
|[24.47392,22.4341...|         0.5564747|[24.47392,22.4341...|
|[23.38084,22.5427...|          0.619608|[23.38084,22.5427...|
|[24.7225799999999...|         0.4611808|[24.7225799999999...|
|[25.2249,24.46752...|         0.6421724|[25.2249,24.46752...|
+--------------------+------------------+--------------------+
only showing top 5 rows



In [ ]:
(trainingData, testData) = data.randomSplit([0.6, 0.4])

In [ ]:
from pyspark.ml.regression import RandomForestRegressor


In [ ]:
rf = RandomForestRegressor(featuresCol="indexedFeatures",numTrees = 3)
pipeline = Pipeline(stages=[featureIndexer, rf])

In [ ]:
import time
start_time = time.process_time()
model = pipeline.fit(trainingData)
end_time = time.process_time()

delta_time = end_time - start_time
print(f'run-time: {round(delta_time/60.0, 2)}')


run-time: 0.03


In [ ]:
predictions = model.transform(testData)

NameError: ignored

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(predictions)
print("R2 on test data = %g" % rmse)

NameError: ignored

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)
print("mae on test data = %g" % rmse)

mae on test data = 0.216453


In [ ]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("rmse on test data = %g" % rmse)

rmse on test data = 0.462042


In [ ]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])
end_time = time.process_time()

delta_time = end_time - start_time
print(f'run-time: {round(delta_time/60.0, 2)}')


TypeError: ignored

In [ ]:
import time
start_time = time.process_time()
model = pipeline.fit(trainingData)
end_time = time.process_time()

delta_time = end_time - start_time
print(f'run-time: {round(delta_time/60.0, 2)}')


In [ ]:
pred = model.transform(testData)

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(pred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.462959


In [ ]:
pred.select("prediction", "label", "features").show(5)

+-------------------+---------+--------------------+
|         prediction|    label|            features|
+-------------------+---------+--------------------+
|0.08997352347977046| 1.737357|[17.21431,17.1692...|
|0.08997352347977046|0.2606139|[17.21571,17.1690...|
|0.08997352347977046|0.6309952|[17.21578,16.6898...|
|0.08997352347977046|0.2155446|[17.2205700000000...|
|0.08997352347977046| 1.286235|[17.22269,17.0582...|
+-------------------+---------+--------------------+
only showing top 5 rows



In [ ]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(pred)
print("R2 on test data = %g" % rmse)

R2 on test data = 0.59736
